In [2]:
import os
import json
import requests
import datetime
from dotenv import load_dotenv
from pytrends.request import TrendReq
import matplotlib.pyplot as plt

# --- API 키 설정 ---
load_dotenv()

# [v17] 네이버 API 키 로드 (pdf_tools.py에서 이관)
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
if not NAVER_CLIENT_ID or not NAVER_CLIENT_SECRET:
    print("[trend_search] 경고: .env 파일에 NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET이 없습니다.")
    print("    (네이버 실시간 트렌드 분석이 불가능합니다)")

In [ ]:
import os
import json
import requests
import datetime
from dotenv import load_dotenv
import time
import random
import json
import datetime
import requests


load_dotenv()

# [v17] 네이버 API 키 로드 (pdf_tools.py에서 이관)
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")
if not NAVER_CLIENT_ID or not NAVER_CLIENT_SECRET:
    print("[trend_search] 경고: .env 파일에 NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET이 없습니다.")
    print("    (네이버 실시간 트렌드 분석이 불가능합니다)")

# ===================================================================
# 2) NAVER DATALAB (안정화)
# ===================================================================
def get_naver_datalab(keywords_list):
    print(f"\n[NaverDataLab] 분석 시작: {keywords_list}")


    url = "https://openapi.naver.com/v1/datalab/search"

    today = datetime.date.today()
    one_year_ago = today - datetime.timedelta(days=360)

    body = {
        "startDate": one_year_ago.strftime("%Y-%m-%d"),
        "endDate": today.strftime("%Y-%m-%d"),
        "timeUnit": "week",
        "keywordGroups": [
            {
                "groupName": "크리스마스",
                "keywords": keywords_list
            }
        ]
    }

    # ---------------------------------------------------
    # 핵심 포인트: UTF-8로 직접 인코딩해서 보내기
    # ---------------------------------------------------
    json_bytes = json.dumps(body, ensure_ascii=False).encode("utf-8")

    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET,
        "Content-Type": "application/json; charset=utf-8"
    }

    try:
        response = requests.post(
            url,
            headers=headers,
            data=json_bytes,   # 🔥 UTF-8 바이트 직접 전달
            timeout=10
        )

        # 한글 응답도 처리 가능하게 설정
        response.encoding = "utf-8"

        data = response.json()

        if "error" in data or "errorCode" in data:
            return {"error": f"Naver API 오류: {data}"}

        print("  ✔ Naver DataLab 분석 완료")
        return data

    except Exception as e:
        print(f"  ❗ Naver DataLab 오류: {e}")
        return {"error": f"Naver DataLab 오류: {e}"}

# ===================================================================
# 3) 통합 테스트용 함수
# ===================================================================
def test_trend_v18():
    print("\n================= TREND TEST v18 =================")
    keywords_list = ["크리스마스", "산타", "겨울축제", "트리장식", "크리스마스이브"]

    datalab_json = get_naver_datalab(keywords_list)
    # Naver DataLab
    print("\n[Naver DataLab 결과]\n", get_naver_datalab)

# 실행
if __name__ == "__main__":
    test_trend_v18()



================= TREND TEST v18 =================

[NaverDataLab] 분석 시작: ['크리스마스', '산타', '겨울축제', '트리장식', '크리스마스이브']
  ✔ Naver DataLab 분석 완료

[Naver DataLab 결과]
 <function get_naver_datalab at 0x000001AAFEE365C0>


In [8]:
import time
import random
import pandas as pd
from typing import List, Dict, Any
from pytrends.request import TrendReq

# ---------------------------
# Pytrends 초기화
# ---------------------------
_PT = TrendReq(
    hl="ko-KR",
    tz=540,
    retries=3,
    backoff_factor=0.1,
)


# ---------------------------
# 관련 검색어 변환 헬퍼
# ---------------------------
def _safe_related_to_list(df):
    if df is None or not hasattr(df, "to_dict"):
        return []
    return df.to_dict("records")


# ---------------------------
# Google Trends 조회 함수
# ---------------------------
def fetch_google_trends(
    keywords: List[str],
    timeframe: str = "today 3-m",
    geo: str = "KR",
    include_related: bool = True,
) -> Dict[str, Any]:

    try:
        _PT.build_payload(
            kw_list=keywords,
            timeframe=timeframe,
            geo=("" if geo.upper() == "GLOBAL" else geo),
        )

        # -----------------------------
        # interest_over_time (429 방어 포함)
        # -----------------------------
        iot = None
        for i in range(4):
            try:
                iot = _PT.interest_over_time()
                break
            except Exception as e:
                if "429" in str(e):
                    time.sleep((2 ** i) + random.uniform(0, 0.5))
                    continue
                return {"iot": pd.DataFrame(), "related": {}, "error": str(e)}

        if iot is None or iot.empty:
            return {"iot": pd.DataFrame(), "related": {}, "error": "빈 결과"}

        # isPartial 제거
        iot = iot.drop(
            columns=[c for c in iot.columns if "ispartial" in str(c).lower()],
            errors="ignore",
        )
        iot.index.name = "date"

        # -----------------------------
        # 관련 검색어
        # -----------------------------
        related_summary = {}
        if include_related:
            try:
                rq = _PT.related_queries() or {}
                if not isinstance(rq, dict):
                    rq = {}

                for kw, data in rq.items():
                    top = data.get("top") if isinstance(data, dict) else None
                    rising = data.get("rising") if isinstance(data, dict) else None
                    related_summary[kw] = {
                        "top": _safe_related_to_list(top),
                        "rising": _safe_related_to_list(rising),
                    }
            except Exception as e:
                related_summary = {"error": str(e)}

        return {"iot": iot, "related": related_summary}

    except Exception as e:
        return {"iot": pd.DataFrame(), "related": {}, "error": str(e)}


# ---------------------------
# 테스트 실행
# ---------------------------
if __name__ == "__main__":
    print("=== Google Trends Test (uv 환경) ===")

    result = fetch_google_trends(["크리스마스", "산타"])

    print("\n[IOT]")
    print(result["iot"].head() if isinstance(result["iot"], pd.DataFrame) else result["iot"])

    print("\n[RELATED]")
    print(result["related"])

    if "error" in result:
        print("\n[ERROR]")
        print(result["error"])


=== Google Trends Test (uv 환경) ===


c:\final_project\ACC\acc-ai\.venv\Lib\site-packages\pytrends\request.py:259: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)



[IOT]
            크리스마스  산타
date                 
2025-08-17     16  18
2025-08-18      9  15
2025-08-19     16  23
2025-08-20     19  16
2025-08-21     18  17

[RELATED]
{'크리스마스': {'top': [{'query': '크리스마스 트리', 'value': 100}, {'query': '크리스마스 마켓', 'value': 60}, {'query': '메리 크리스마스', 'value': 45}, {'query': '크리스마스 선물', 'value': 32}, {'query': '크리스마스 영어', 'value': 29}, {'query': '캐롤', 'value': 28}, {'query': '크리스마스 악몽', 'value': 28}, {'query': '크리스마스 캐롤', 'value': 27}, {'query': 'christmas', 'value': 27}, {'query': '더 현대 크리스마스', 'value': 27}, {'query': '크리스마스 장식', 'value': 26}, {'query': '크리스마스 영화', 'value': 26}, {'query': '크리스마스 케이크', 'value': 25}, {'query': '크리스마스 노래', 'value': 24}, {'query': '크리스마스 예약', 'value': 23}, {'query': '크리스마스 의 악몽', 'value': 23}, {'query': '겨울', 'value': 23}, {'query': '산타', 'value': 21}, {'query': '크리스마스 배경', 'value': 19}, {'query': '크리스마스 게임', 'value': 18}, {'query': '크리스마스 영어 로', 'value': 17}, {'query': '크리스마스 일러스트', 'value': 17}, {'query': '크리스마스 호텔', 'v

In [6]:
# ============================================================
# 📌 네이버 데이터랩 + 구글트렌드 1년 분석 (그래프 데이터만 반환)
# ============================================================
def get_trend_data_1year(keywords: list):
    """
    프론트에서 그래프 그릴 수 있도록:
      - naver_datalab.weekly_ratio (최근 1년간)
      - google_trends.interest_over_time (최근 1년간)
      - google_trends.related.top/rising 10개씩
    만 깔끔한 JSON으로 반환한다.
    """

    # ------------------------------------------------------------
    # 1) NAVER DataLab (1년 기준)
    # ------------------------------------------------------------
    naver_data = get_naver_datalab(keywords)

    naver_week_list = []
    try:
        raw = naver_data["results"][0]["data"]
        for item in raw:
            naver_week_list.append({
                "period": item["period"],     # '2024-10-30'
                "ratio": item["ratio"]        # 키워드 묶음 검색량
            })
    except:
        naver_week_list = []
    

    # ------------------------------------------------------------
    # 2) Google Trends (1년 기준)
    # ------------------------------------------------------------
    google_result = fetch_google_trends(
        keywords=keywords,
        timeframe="today 12-m",   # 1년
        include_related=True
    )

    google_iot = []
    try:
        df = google_result["iot"]
        for date, row in df.iterrows():
            google_iot.append({
                "date": str(date.date()),
                **{col: int(row[col]) for col in df.columns}
            })
    except:
        google_iot = []

    # ------------------------------------------------------------
    # 3) Google Related Queries (TOP 10 / RISING 10)
    # ------------------------------------------------------------
    google_related = {}
    try:
        rq = google_result["related"]
        for kw, items in rq.items():
            google_related[kw] = {
                "top": items.get("top", [])[0:10],
                "rising": items.get("rising", [])[0:10]
            }
    except:
        google_related = {}

    # ------------------------------------------------------------
    # 4) 최종 JSON 구조
    # ------------------------------------------------------------
    return {
        "keywords": keywords,
        "naver": {
            "weekly": naver_week_list
        },
        "google": {
            "iot": google_iot,
            "related": google_related
        }
    }


# ============================================================
# 📌 단독 테스트 실행
# ============================================================
if __name__ == "__main__":
    keywords = ["크리스마스", "산타", "겨울축제", "트리장식", "크리스마스이브"]
    result = get_trend_data_1year(keywords)
    print(json.dumps(result, ensure_ascii=False, indent=2))



[NaverDataLab] 분석 시작: ['크리스마스', '산타', '겨울축제', '트리장식', '크리스마스이브']
  ✔ Naver DataLab 분석 완료


c:\final_project\ACC\acc-ai\.venv\Lib\site-packages\pytrends\request.py:259: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


{
  "keywords": [
    "크리스마스",
    "산타",
    "겨울축제",
    "트리장식",
    "크리스마스이브"
  ],
  "naver": {
    "weekly": [
      {
        "period": "2024-11-18",
        "ratio": 2.08992
      },
      {
        "period": "2024-11-25",
        "ratio": 2.96461
      },
      {
        "period": "2024-12-02",
        "ratio": 3.32766
      },
      {
        "period": "2024-12-09",
        "ratio": 3.56938
      },
      {
        "period": "2024-12-16",
        "ratio": 5.35004
      },
      {
        "period": "2024-12-23",
        "ratio": 100
      },
      {
        "period": "2024-12-30",
        "ratio": 0.76963
      },
      {
        "period": "2025-01-06",
        "ratio": 0.74972
      },
      {
        "period": "2025-01-13",
        "ratio": 0.67717
      },
      {
        "period": "2025-01-20",
        "ratio": 0.5037
      },
      {
        "period": "2025-01-27",
        "ratio": 0.45795
      },
      {
        "period": "2025-02-03",
        "ratio": 0.34593
      },
    